In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 5 – Imputation

## DSC 80, Spring 2022

### Due Date: Monday, May 2nd at 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying `lab.py` file will be tested (a la DSC 20),
2. The notebook may be graded (if it contains free response questions or asks you to draw plots).

**Do not change the function names in the `lab.py` file!**
- The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or google around for `git revert`.

**Tips for working in the notebook**:
- The notebooks serve to present the questions and give you a place to present your results for later review.
- The notebooks in *lab assignments* are not graded (only the `lab.py` file is submitted and graded).
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file.

**Tips for developing in the `lab.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional helper functions to solve the lab! 
- Always document your code!

### Importing code from `lab.py`

* We import our `lab.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from lab import *

In [2]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

import requests
import bs4

***Note:*** While working on the lab, check the Campuswire post titled "Lab 5 Released!" for any clarifications.

## Question 1 – Payment Data 💰

In `data/payment.csv`, you are given a dataset of payment information for purchases made on January 1st, 2019. The dataset contains the purchaser's `'Id'`, `'credit_card_type'`, `'credit_card_number'`, and `'date_of_birth'`.

Your job is to assess the missingness in the payments data. The question you need to answer is, **is `'credit_card_number'` missing at random dependent on the age of the shopper?**. Let's approach this question in two ways.

#### `first_round`

Look at distribution of ages by missingness of `'credit_card_number'` and determine if the missingness is dependent on age or not.

Use the following steps to approach this problem:

- Compute the ages of the purchasers.
- Draw the distribution of ages by missingness of `'credit_card_number'`. Specifically, you will draw two histograms or density curves:
    - One of ages where `'credit_card_number'` is missing.
    - One of ages where `'credit_card_number'` is not missing.
- Perform a permutation test for whether or not the two distributions mentioned above are drawn from the same population distribution. Use a 5% significance level. Use the **absolute difference of means** as your test statistic.

Write a function `first_round` with no arguments that returns a **list** with two values:
* The first value is the p-value from your permutation test. 
* The second value is either `'R'` if you reject the null hypothesis, or `'NR'` if you fail to reject the null.

**Does the result match your guess? If not, what might be a problem?**

***Hint:*** 
- Look at [Lecture 12](https://dsc80.com/resources/lectures/lec12/lec12.html#Assessing-missingness-through-data).
- To find a customer's age, compute the number of years between their birth year and 2022.

<br>

#### `second_round`

Run another permutation test for the empirical distribution of age conditional on `'credit_card_number'` with a 5% significance level. This time, use the **Kolmogorov-Smirnov statistic** as your test statistic.

Write a function `second_round` with no arguments that returns a __list__ with three values: 
* The first value is the p-value from your new permutation test.
* The second value is either `'R'` if you reject the null hypothesis, or `'NR'` if you fail to reject the null. 
* The third value is your final conclusion: `'D'` (the missingness of `'credit_card_number'` is dependent on age) or `'ND'` (the missingness of `'credit_card_number'` is not dependent on age).

***Hint:*** 
- In both [Lecture 12](https://dsc80.com/resources/lectures/lec12/lec12.html#Assessing-missingness-through-data) and [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html), we ran permutation tests using the Kolmogorov-Smirnov test statistic **without `for`-loops**. You can use this same procedure; we have already imported `stats` from `scipy`.

In [30]:
def extract_byear(date):
    if type(date) == float:
        return np.nan
    else:
        return int(date[7:])

In [45]:
# Run your permutation tests in the Jupyter Notebook;
# put your final results in lab.py
payments_fp = os.path.join('data', 'payment.csv')
payments = pd.read_csv(payments_fp)
payments.head()


,id,credit_card_type,credit_card_number,date_of_birth
0,1,diners-club-enroute,2.018706e+14,25-Sep-1982
1,2,americanexpress,3.737511e+14,08-Jan-1946
2,3,jcb,3.570296e+15,NaN
3,4,mastercard,5.318327e+15,NaN
4,5,maestro,6.759827e+17,20-Apr-1975


In [46]:
payments['date_of_birth'] = payments['date_of_birth'].apply(extract_byear)
#payments['date_of_birth'][3]
payments['Age'] = 2022 - payments['date_of_birth'] 
payments
payments['Credit_missing'] = payments['credit_card_number'].isna()
payments

,id,credit_card_type,credit_card_number,date_of_birth,Age,Credit_missing
0,1,diners-club-enroute,2.018706e+14,1982.0,40.0,False
1,2,americanexpress,3.737511e+14,1946.0,76.0,False
2,3,jcb,3.570296e+15,NaN,NaN,False
3,4,mastercard,5.318327e+15,NaN,NaN,False
4,5,maestro,6.759827e+17,1975.0,47.0,False
...,...,...,...,...,...,...
995,996,visa-electron,NaN,1987.0,35.0,True
996,997,jcb,3.570380e+15,1994.0,28.0,False
997,998,mastercard,5.010121e+15,1961.0,61.0,False
998,999,visa-electron,NaN,NaN,NaN,True


In [50]:
(1 + np.nan) / 2

nan

In [89]:
shuffled = payments.copy()
shuffled['Credit_missing'] = shuffled['credit_card_number'].isna()

n_repetitions = 100
means1 = []

for _ in range(n_repetitions):
    
    # Shuffling genders and assigning back to the DataFrame
    shuffled['Credit_missing'] = np.random.permutation(shuffled['Credit_missing'])
    
    test_stat1 = np.abs(np.nanmean(shuffled['Age'][shuffled['Credit_missing'].values].values) - np.nanmean(shuffled['Age'][~shuffled['Credit_missing'].values].values))
    
    means1.append(test_stat1)
   

In [54]:
means1

[1.2663537962769311,
 2.0325954376767967,
 0.9815734989648064,
 1.8890051075853052,
 0.41189427312774995,
 1.9062374622194866,
 3.6740659873847648,
 0.8865180970444158,
 0.6807564698075623,
 1.0388370289137896,
 0.1519512129884717,
 4.729736935655886,
 1.1102412828076567,
 0.3923273657288959,
 0.5991226079721628,
 0.059309309309306,
 0.12010839838894327,
 4.006260143751447,
 0.7337947205203861,
 0.7145451735064512,
 3.4760594814834036,
 2.8405370843989743,
 0.8437611091361532,
 4.141252552756974,
 0.6812212908416129,
 2.123711340206185,
 1.1840669378608126,
 1.7475292825768634,
 3.7942505191088216,
 2.518840579710144,
 0.8953258393210319,
 4.467076281010897,
 1.532884360213309,
 1.6415623772377828,
 4.7129795396419425,
 0.38686721656153367,
 1.648956817079089,
 0.3916253600809654,
 1.7054173005868236,
 0.4198364735158151,
 1.7444146431934229,
 2.992464641780657,
 0.6649616368286431,
 1.3033447662919642,
 0.2687292555713583,
 3.8584979108815816,
 1.3001712817584874,
 1.9418793139510555,

In [93]:
from scipy.stats import ks_2samp

In [90]:
observed = np.abs(np.nanmean(payments['Age'][payments['credit_card_number'].isna().values].values) - np.nanmean(shuffled['Age'][~payments['credit_card_number'].isna().values].values))
observed

2.940722967870805

In [91]:
(means1 >= observed).mean()

0.13

In [134]:
payments

,id,credit_card_type,credit_card_number,date_of_birth,Age,Credit_missing
0,1,diners-club-enroute,2.018706e+14,1982.0,40.0,False
1,2,americanexpress,3.737511e+14,1946.0,76.0,False
2,3,jcb,3.570296e+15,NaN,NaN,False
3,4,mastercard,5.318327e+15,NaN,NaN,False
4,5,maestro,6.759827e+17,1975.0,47.0,False
...,...,...,...,...,...,...
995,996,visa-electron,NaN,1987.0,35.0,True
996,997,jcb,3.570380e+15,1994.0,28.0,False
997,998,mastercard,5.010121e+15,1961.0,61.0,False
998,999,visa-electron,NaN,NaN,NaN,True


In [113]:
obs = ks_2samp(
    payments.groupby('Credit_missing')['Age'].get_group(True),
    payments.groupby('Credit_missing')['Age'].get_group(False)
)
obs

KstestResult(statistic=0.12699202780883062, pvalue=0.0344518152440148)

In [117]:
n_repetitions = 500
shuffled = payments.copy()

ks_stats = []
for _ in range(n_repetitions):
    
    # Shuffling the data and assigning it back to the DataFrame
    shuffled['Credit_missing'] = np.random.permutation(shuffled['Credit_missing'])
    
    # Computing and storing the K-S statistic
    groups = shuffled.groupby('Credit_missing')['Age']
    ks_stat = ks_2samp(groups.get_group(True), groups.get_group(False)).statistic
    ks_stats.append(ks_stat)
    

In [111]:
obs.statistic

0.12699202780883062

In [118]:
np.mean(ks_stats >= obs.pvalue)

0.986

In [119]:
# don't change this cell, but do run it -- it is needed for the tests to work
first_pval, first_result = first_round()
second_pval, second_result, second_result1 = second_round()

In [120]:
grader.check("q1")

q1 results: All test cases passed!

## Question 2 – Missing Heights 🕵️

In the file `data/missing_heights.csv` are the heights of children and their fathers (`'child'` and `'father'`). The `'child_X'` columns are missing values in varying proportions; for each X, `'child_X'` is X\% not missing (and hence (100-X)\% missing). The missingness of these `'child_X'` columns were created as MAR dependent on `'father'` height (similar to what was done in Lecture 12). The missingness of these `'child_X'` columns are all equally dependent on father height.

You will attempt to **verify** the missingness of the `'child_X'` columns as being dependent on the `'father'` height column by using permutation tests. Your permutation tests should use the Kolmogorov-Smirnov test statistic. You can use `scipy.stats`' built-in K-S function to run your permutation tests and compute your p-values; you don't need to simulate manually using a `for`-loop.

#### `verify_child`

Write a function `verify_child` that takes in the `heights` DataFrame and returns a Series of p-values from your permutation tests, indexed by the columns `'child_X'`.

***Notes:*** 

- Unlike in Question 1, your permutation tests should run within your `verify_child` function.
- You may loop over the **columns** of `heights`.

<br>

#### `missing_data_amounts`

Now, interpret your results. In the function `missing_data_amounts`, return a __list__ of correct statements from the options below:
1. The p-value for `'child_50'` is small because the *sampling distribution* of test statistics has low variance.
2. MAR is hardest to determine when there are very different proportions of missing and not-missing values.
3. The difference between p-values for `'child_5'` and `'child_95'` is due to randomness.
4. We should expect the p-value of `'child_X'` and `'child_(100-X)'` to be similar.
5. We should only expect the p-value of `'child_X'` and `'child_(100-X)'` to be similar if the columns are MCAR.

In [189]:
ser

child_95     4.155249e-80
child_90    1.514759e-130
child_75    3.043878e-227
child_50    5.276715e-280
child_25    3.043878e-227
child_10    1.514759e-130
child_5      4.155249e-80
dtype: float64

In [179]:
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
heights

,child,father,child_95,child_90,child_75,child_50,child_25,child_10,child_5
0,73.2,78.5,73.2,73.2,73.2,NaN,NaN,NaN,NaN
1,69.2,78.5,69.2,69.2,69.2,NaN,NaN,NaN,69.2
2,69.0,78.5,69.0,69.0,69.0,69.0,NaN,NaN,NaN
3,69.0,78.5,69.0,69.0,NaN,69.0,NaN,NaN,NaN
4,73.5,75.5,73.5,73.5,NaN,73.5,73.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...
929,64.0,62.0,64.0,64.0,64.0,64.0,NaN,NaN,NaN
930,62.0,62.0,62.0,62.0,62.0,62.0,62.0,NaN,NaN
931,61.0,62.0,61.0,61.0,61.0,NaN,NaN,NaN,NaN
932,66.5,62.5,66.5,66.5,66.5,66.5,66.5,NaN,NaN


In [ ]:
gpA = heights.loc[data['group'] == 'A', 'data']
gpB = heights.loc[data['group'] == 'B', 'data']

In [180]:
indeces = heights.columns
#ser = pd.Series()
names = []
k_stats = []
for n in indeces:
    if n.find('child_') != -1:
        heights[n + ' missing'] = heights[n].isna()
        
        names.append(n)
        k_stats.append(ks_2samp(
        heights.groupby(n + ' missing')[n].get_group(True),
        heights.groupby(n + ' missing')[n].get_group(False)
        ).pvalue)
        
heights

ser = pd.Series(index = names, data = k_stats)
ser

child_95     4.155249e-80
child_90    1.514759e-130
child_75    3.043878e-227
child_50    5.276715e-280
child_25    3.043878e-227
child_10    1.514759e-130
child_5      4.155249e-80
dtype: float64

In [181]:
names = names[::-1]
k_stats = k_stats[::-1]

In [182]:
k_stats

[4.155248607980582e-80,
 1.5147594924454639e-130,
 3.0438784208535273e-227,
 5.276715361764929e-280,
 3.0438784208535273e-227,
 1.5147594924454639e-130,
 4.155248607980582e-80]

In [172]:
names

['child_5',
 'child_10',
 'child_25',
 'child_50',
 'child_75',
 'child_90',
 'child_95']

In [187]:
k_stats

[4.155248607980582e-80,
 1.5147594924454639e-130,
 3.0438784208535273e-227,
 5.276715361764929e-280,
 3.0438784208535273e-227,
 1.5147594924454639e-130,
 4.155248607980582e-80]

In [190]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
out1_q2 = verify_child(heights.copy())
out2_q2 = missing_data_amounts()

In [178]:
out1_q2['child_50'] < out1_q2['child_95']

False

In [191]:
grader.check("q2")

q2 results: All test cases passed!

## Question 3 – Imputing Heights 🧍📏

In [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html), you learned how to perform single-valued imputation conditionally on a **categorical** column: impute with the mean for each group. That is, for each distinct value of the **categorical** column, there is a single imputed value.

Here, you will perform single-valued imputation conditionally on a **quantitative** column. 

You will work with a version of the `heights` DataFrame, `new_heights`, that has a `'father'` column and a single `'child'` column. The `'child'` column has missing values. To impute the `'child'` column, transform the `'father'` column into a categorical column by binning the values of `'father'` into [quartiles](https://en.wikipedia.org/wiki/Quartile). Once this is done, you can impute `'child'` as in lecture (and described above).

<br>

#### `cond_single_imputation`

Write a function `cond_single_imputation` that takes in a DataFrame with columns `'father'` and `'child'` (where `'child'` has missing values) and performs a single-valued mean imputation of the `'child'` column, conditional on `'father'`. Your function should return a **Series**.

***Hints:*** 
- `pd.qcut` may be helpful!
- The `transform` is useful for this question (see [Lecture 7](https://dsc80.com/resources/lectures/lec07/lec07.html#The-transform-method)), though it's also possible using `aggregate`. 
- As a reminder, *loops are not allowed*, and functions mentioned in "Hints" are not required.

In [193]:
heights_fp = os.path.join('data', 'missing_heights.csv')
new_heights = pd.read_csv(heights_fp)[['father', 'child_50']]
new_heights

,father,child_50
0,78.5,NaN
1,78.5,NaN
2,78.5,69.0
3,78.5,69.0
4,75.5,73.5
...,...,...
929,62.0,64.0
930,62.0,62.0
931,62.0,NaN
932,62.5,66.5


In [225]:
pd.qcut(new_heights['father'],4)

0        (71.0, 78.5]
1        (71.0, 78.5]
2        (71.0, 78.5]
3        (71.0, 78.5]
4        (71.0, 78.5]
            ...      
929    (61.999, 68.0]
930    (61.999, 68.0]
931    (61.999, 68.0]
932    (61.999, 68.0]
933    (61.999, 68.0]
Name: father, Length: 934, dtype: category
Categories (4, interval[float64, right]): [(61.999, 68.0] < (68.0, 69.0] < (69.0, 71.0] < (71.0, 78.5]]

In [206]:
def mean_impute(ser):
    return ser.fillna(ser.mean())

In [ ]:
heights_mar_cat.groupby('gender')['child'].transform(mean_impute)

In [209]:
heights_fp = os.path.join('data', 'missing_heights.csv')
new_heights = pd.read_csv(heights_fp)[['father', 'child_50']]
new_heights = new_heights.rename(columns={'child_50': 'child'})

In [213]:
heights_fp = os.path.join('data', 'missing_heights.csv')
new_heights = pd.read_csv(heights_fp)[['father', 'child_50']]
new_heights = new_heights.rename(columns={'child_50': 'child'})
new_heights['cuts'] = pd.qcut(new_heights['father'],4)
new_heights

,father,child,cuts
0,78.5,NaN,"(71.0, 78.5]"
1,78.5,NaN,"(71.0, 78.5]"
2,78.5,69.0,"(71.0, 78.5]"
3,78.5,69.0,"(71.0, 78.5]"
4,75.5,73.5,"(71.0, 78.5]"
...,...,...,...
929,62.0,64.0,"(61.999, 68.0]"
930,62.0,62.0,"(61.999, 68.0]"
931,62.0,NaN,"(61.999, 68.0]"
932,62.5,66.5,"(61.999, 68.0]"


In [214]:
new_heights.groupby('cuts')['child'].mean()

cuts
(61.999, 68.0]    65.481383
(68.0, 69.0]      66.933750
(69.0, 71.0]      67.434307
(71.0, 78.5]      68.083871
Name: child, dtype: float64

In [221]:
new_heights['cuts'] = pd.qcut(new_heights['father'],4)
ser = new_heights.groupby('cuts')['child'].transform(mean_impute)#[:50]
ser

0      68.083871
1      68.083871
2      69.000000
3      69.000000
4      73.500000
         ...    
929    64.000000
930    62.000000
931    65.481383
932    66.500000
933    65.481383
Name: child, Length: 934, dtype: float64

In [201]:
np.quantile(new_heights['father'],0.25)

68.0

In [226]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
new_heights = pd.read_csv(heights_fp)[['father', 'child_50']]
new_heights = new_heights.rename(columns={'child_50': 'child'})
out_q3 = cond_single_imputation(new_heights.copy())

In [227]:
grader.check("q3")

q3 results: All test cases passed!

## Question 4 – Probabilistic Imputation 🎲

In [Lecture 13](https://dsc80.com/resources/lectures/lec13/lec13.html#Probabilistic-imputation), you learned how to impute a quantitative column by sampling from the observed values. **One problem with this technique is that the imputation will never generate imputed values that weren't already in the dataset.** For example, 57, 57.5, and 59 are values in the `'child'` column of `new_heights` while 58 is not. Thus, any imputation done by sampling from the observed values in the `'child'` column will not be able to generate a height of 58, even though it's clearly a reasonable value to occur in the dataset.

To keep things simple, you will impute the `'child'` column **unconditionally** from the distribution of `'child'` heights present in the dataset. This means that you will use the values present in `'child'` to impute missing values, without looking at other columns.

An approach to quantitative imputation that overcomes the limitation mentioned above is as follows:
- Create a histogram of observed `'child'` heights, using 10 bins.
- Use the histogram to generate a number within the observed range of `'child'` heights:
    - The likelihood a generated number belongs to a given bin is equal to the area of that bin. (Remember, in histograms, areas are proportions.)
    - Any number within a fixed bin is equally likely to occur.
    
Let's illustrate this approach with an example. Let `demo` be the array of 10 numbers defined below.

```py
demo = np.array([10, 11, 11, 13, 14, 14, 13.5, 14, 15, 16])
```

- The first step is creating a histogram of `demo`. Note that with this small dataset, we will use 3 bins, but you will be using 10 bins in your imputation process.

<img src='imgs/demo_histogram.png' width=300>

- Note that in your process, you don't actually need to draw a histogram – instead, use `np.histogram`.
- In the histogram above, we see that $2 \cdot 0.15 = 0.3 = 30\%$ of values lie in the [10, 12) bin, $2 \cdot 0.1 = 0.2 = 20\%$ of values lie in the [12, 14) bin, and $2 \cdot 0.25 = 0.5 = 50\%$ of values lie in the [14, 16] bin.
- Next, we need to pick a bin at random. There's a 30\% chance we pick the [10, 12) bin, a 20\% chance we pick the [12, 14) bin, and a 50\% chance we pick the [14, 16] bin. `np.random.choice` will be helpful in picking a bin at random.
- Once we pick a bin, we pick a number **uniformly at random** from within the bin. For instance, suppose we randomly chose the [14, 16] bin in the previous step. We then must select a (real) number between 14 and 16 uniformly at random. `np.random.uniform` can help you here.

<br>

#### `quantitative_distribution`
    
Create a function `quantitative_distribution` that takes in a Series, `child`, in which some values are missing, and an integer `N > 0`, and returns an **array** of `N` imputed values using the method described above. 

***Note:*** You may use a `for`-loop.

<br>

#### `impute_height_quant`

Create a function `impute_height_quant` that takes in a Series, `child`, in which some values are missing and imputes them using the scheme above. `impute_height_quant` should return a Series that is the same length of `child` but with no missing values. **You should use `quantitative_distribution` to help you do this.**

In [311]:
hist

array([  3,  15,  54, 110,  81,  85,  99,  16,   3,   1])

In [310]:
p=hist/hist.sum()
p

array([0.00642398, 0.03211991, 0.11563169, 0.23554604, 0.17344754,
       0.18201285, 0.21199143, 0.03426124, 0.00642398, 0.00214133])

In [239]:
heights

,child,father,child_95,child_90,child_75,child_50,child_25,child_10,child_5
0,73.2,78.5,73.2,73.2,73.2,NaN,NaN,NaN,NaN
1,69.2,78.5,69.2,69.2,69.2,NaN,NaN,NaN,69.2
2,69.0,78.5,69.0,69.0,69.0,69.0,NaN,NaN,NaN
3,69.0,78.5,69.0,69.0,NaN,69.0,NaN,NaN,NaN
4,73.5,75.5,73.5,73.5,NaN,73.5,73.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...
929,64.0,62.0,64.0,64.0,64.0,64.0,NaN,NaN,NaN
930,62.0,62.0,62.0,62.0,62.0,62.0,62.0,NaN,NaN
931,61.0,62.0,61.0,61.0,61.0,NaN,NaN,NaN,NaN
932,66.5,62.5,66.5,66.5,66.5,66.5,66.5,NaN,NaN


In [250]:
heights['child_50']

0       NaN
1       NaN
2      69.0
3      69.0
4      73.5
       ... 
929    64.0
930    62.0
931     NaN
932    66.5
933     NaN
Name: child_50, Length: 934, dtype: float64

In [308]:
demo = heights['child_50'].dropna()
probs = []
N =100
for n in np.arange(N):
    hist, edges = np.histogram(demo, bins=10)
    #hist
    #edges
    index = np.random.choice(a=[0,1,2,3,4,5,6,7,8,9], p=hist/hist.sum())
    probs.append(np.random.uniform(edges[index], edges[index+1]))
    
probs

[70.07826639529797,
 62.980739256833886,
 67.18444565070082,
 64.90741754626394,
 70.85537474122388,
 62.25411479558417,
 70.56071617579961,
 70.79827210493248,
 61.686829841839916,
 68.88061900816562,
 71.88384321670105,
 66.35515388782699,
 73.740640859066,
 67.42106216244505,
 66.68848818610644,
 69.27770142025678,
 71.17838103434266,
 66.99199963264772,
 61.49620146293728,
 68.42375037689274,
 61.82790539050011,
 63.74266161999201,
 69.44529520867486,
 77.23541650029756,
 71.40392850157997,
 60.73796198142112,
 68.65740832424083,
 64.18133439697633,
 67.736989433088,
 66.1828379860519,
 71.20014245163034,
 66.2026233424181,
 61.95437179620629,
 71.15159562179684,
 71.91662641683975,
 58.514351039771284,
 69.48284801262508,
 64.04705901955364,
 70.45837684440696,
 70.72314862180257,
 65.86576585288468,
 70.43603152301836,
 59.706642453165166,
 73.72898500567732,
 65.00214319043162,
 64.20593010204112,
 69.49807571023018,
 68.77810564383785,
 61.51696520009036,
 66.39345634549333,
 7

In [260]:
hist/hist.sum()

array([0.00642398, 0.03211991, 0.11563169, 0.23554604, 0.17344754,
       0.18201285, 0.21199143, 0.03426124, 0.00642398, 0.00214133])

In [321]:
out = quantitative_distribution(heights['child_50'],100)
np.isclose(out.std(), 3.5, atol=0.65)

True

In [362]:
heights['child_50']

0       NaN
1       NaN
2      69.0
3      69.0
4      73.5
       ... 
929    64.0
930    62.0
931     NaN
932    66.5
933     NaN
Name: child_50, Length: 934, dtype: float64

In [370]:
ser = heights['child_50']
missing = heights['child_50'].isna().sum()
missing
out = quantitative_distribution(heights['child_50'],missing)
out

array([68.94810864, 69.87437343, 70.15140315, 70.22255209, 70.27412561,
       65.80525835, 63.50214171, 71.53575737, 71.75136091, 64.62275085,
       66.15914137, 63.66134586, 71.87243996, 64.27831216, 67.71174897,
       68.5692397 , 69.55388693, 64.06712093, 66.3753672 , 60.73606001,
       65.685719  , 62.96210567, 66.5088366 , 70.12238731, 67.49363938,
       78.8839469 , 67.45530833, 64.06918367, 60.93471734, 63.23698229,
       71.52205961, 70.41079117, 70.98846298, 66.40570101, 67.5121258 ,
       62.22043502, 68.55355951, 68.25394908, 65.78962919, 64.29333822,
       71.1954316 , 65.12763339, 66.53744068, 66.08129467, 69.46257377,
       63.10815267, 64.55782396, 71.62890716, 66.43329319, 64.6485864 ,
       66.41404115, 75.75658152, 63.07022508, 71.05544104, 69.76947417,
       71.96322857, 70.15630264, 71.52149126, 63.27158822, 63.0894732 ,
       67.66853014, 63.49858972, 69.23818034, 67.61584483, 61.33239116,
       63.24083218, 63.92913872, 63.52940326, 70.6799695 , 68.11

In [371]:

ser.update(pd.Series(out, index = ser.index[ser.isna()]))
ser

0      68.948109
1      69.874373
2      69.000000
3      69.000000
4      73.500000
         ...    
929    64.000000
930    62.000000
931    63.348629
932    66.500000
933    59.016169
Name: child_50, Length: 934, dtype: float64

In [359]:
ser.isna().sum()

0

In [369]:
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
heights

,child,father,child_95,child_90,child_75,child_50,child_25,child_10,child_5
0,73.2,78.5,73.2,73.2,73.2,NaN,NaN,NaN,NaN
1,69.2,78.5,69.2,69.2,69.2,NaN,NaN,NaN,69.2
2,69.0,78.5,69.0,69.0,69.0,69.0,NaN,NaN,NaN
3,69.0,78.5,69.0,69.0,NaN,69.0,NaN,NaN,NaN
4,73.5,75.5,73.5,73.5,NaN,73.5,73.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...
929,64.0,62.0,64.0,64.0,64.0,64.0,NaN,NaN,NaN
930,62.0,62.0,62.0,62.0,62.0,62.0,62.0,NaN,NaN
931,61.0,62.0,61.0,61.0,61.0,NaN,NaN,NaN,NaN
932,66.5,62.5,66.5,66.5,66.5,66.5,66.5,NaN,NaN


In [329]:
heights[heights['child_50'].isna() == True].update(out)
heights['child_50']

0      73.026297
1      67.110077
2      69.507603
3      64.700070
4      64.059559
         ...    
929    64.000000
930    62.000000
931          NaN
932    66.500000
933          NaN
Name: child_50, Length: 934, dtype: float64

In [339]:
heights[heights['child_50'].isna()]['child_50']

0     NaN
1     NaN
5     NaN
6     NaN
8     NaN
       ..
921   NaN
922   NaN
928   NaN
931   NaN
933   NaN
Name: child_50, Length: 467, dtype: float64

In [372]:
# don't change this cell, but do run it -- it is needed for the tests to work
heights_fp = os.path.join('data', 'missing_heights.csv')
heights = pd.read_csv(heights_fp)
child = heights['child_50']
quantitative_distribution_out_q4 = quantitative_distribution(child.copy(), 100)
impute_height_quant_out_q4 = impute_height_quant(child.copy())

In [373]:
grader.check("q4")

q4 results: All test cases passed!

## Question 5 – The Rules of Web Scraping 🚔

In Lecture 14, we will start studying web scraping. This question will give you an introduction to the rules behind it.

Many sites have a published policy allowing or disallowing automatic access to their site. Often, this policy is in a text file named `robots.txt`. [Here is a good article](https://moz.com/learn/seo/robotstxt) that explains what these files are, where to find them, and how to use them. **After reading the article**, please answer the following questions.

<br>

### Multiple-Choice Questions

**1. What is the purpose of `robots.txt`?**

1. To informs agents which pages to crawl.

2. To informs agents that the site is automated.

3. To inform agents that robots will chase them down if their info is stolen.


**2. Where do you put your `robots.txt` file?**

1. In the folder you want to disallow.

2. In the root directory of your website.

3. In a Google search.

**3. Is it illegal to scrape a site if there is no `robots.txt` present?** (***Hint:*** [Read this](https://medium.com/@tjwaterman99/web-scraping-is-now-legal-6bf0e5730a78) if you're not sure. If you aren't able to access the article because Medium says "You've read all your free member-only stories...", try opening the article in a new incognito or private browsing window 😅.)

1. Yes.

2. No.

**4. Can each subdomain on a root domain use separate `robots.txt` files?**

1. Yes.

2. No.


<br>


### Website Hunt

Next, **find three websites that explicitly use a `robots.txt` file and allow scraping by everyone, and three that do not allow scraping by generic user-agents**.

When browsing through `robots.txt` files, you may notice that some have entries for several different user-agents. The user-agent you are interested in for this question is the generic user-agent, which is denoted by `*`. A `/` after a `Disallow:` indicates that no scraping is allowed, whereas nothing after the `Disallow:` indicates that all scraping is allowed.

***Notes:***
- During your search, you may notice that very few websites allow scraping by everyone. When trying to find websites that satisfy this criteria, you are encouraged to think contextually about what kinds of websites would and wouldn't mind you scraping their data. Would a government website likely mind scraping by everyone? How about a website someone created just for fun?

- Some websites may cause Gradescope to time out. Please change a website if you encounter this issue. 

- Below, you are asked to store the URLs of the websites you find in a list. When storing URLs, you can add `'/robots.txt'` to the end, but you don't have to – either format will be accepted.

<br>


#### `answers`

Create an argument-free function `answers` that returns **two lists**:

* one containing your answers to the multiple-choice questions, and
* one containing the URLs of the sites you found. The first 3 URLs in the list should be of websites that allow scraping by everyone, and the last 3 URLs in the list should be of websits that disallow scraping by generic user-agents.

In [374]:
# don't change this cell, but do run it -- it is needed for the tests to work
mc_answers, websites = answers()

In [375]:
grader.check("q5")

q5 results: All test cases passed!

## Congratulations! You're done! 🏁

Submit your `lab.py` file to Gradescope. Note that you only need to submit the `lab.py` file; this notebook should not be uploaded.

Before submitting, you should ensure that all of your work is in the `lab.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `lab.py` file. Run the cell below; you should see no output.

In [376]:
!python -m doctest lab.py

In addition, `grader.check_all()` will verify that your work passes the public tests.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [377]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!